### MILESTONE 3. ERROR ESTIMATOR OF NUMERICAL SOLUTIONS

In [ ]:
import numpy as np


import numpy as np
from numpy.linalg import norm  

def cauchy_integration(F, U0, T, N, method):
    
    U0 = np.array(U0, dtype=float).flatten()

    dimension_U0 = len(U0)

    Delta_t = T/N
    U_solucion = np.zeros((N+1, dimension_U0))
    U_solucion[0,:]= U0 
    vector_tiempos= np.linspace(0, T, N+1)

    maxiter = 100 # Para el Crank-Nicholson
    tol = 1e-10   # Para Crank-Nicholson

    for n in range(N):

        if method == "euler":

            U_solucion[n+1, :] = U_solucion[n, :]  + Delta_t*F(U_solucion[n, :], vector_tiempos[n])



        elif method == "rk4":
            
            k1 = F(U_solucion[n, :], vector_tiempos[n])
            k2 = F(U_solucion[n, :] + (Delta_t/2)*k1, vector_tiempos[n] + 0.5* Delta_t)
            k3 = F(U_solucion[n, :] + (Delta_t/2)*k2, vector_tiempos[n] + 0.5* Delta_t)
            k4 = F(U_solucion[n, :] + Delta_t*k3, vector_tiempos[n] + Delta_t)

            U_solucion[n+1, :] = U_solucion[n, :]  + (Delta_t/6)*(k1 + 2*k2 + 2*k3 + k4)


        elif method == "crank_nicholson":


                U_inicial= U_solucion[n,:]

                for k in range(maxiter):
                    
                    F_n = F(U_solucion[n, :], vector_tiempos[n])
                    F_siguiente = F(U_inicial, vector_tiempos[n] + Delta_t)

                
                    # valor de U
                    U_nueva = U_solucion[n,:] + (Delta_t/2)*(F_n+ F_siguiente)

                
                    if np.linalg.norm(U_nueva - U_inicial)<tol:
                        break

                    U_inicial=U_nueva

                U_solucion[n+1,:] = U_nueva




    return vector_tiempos, U_solucion

import matplotlib.pyplot as plt

def F(U, t):
        
            r = U[0:2]
            rd= U[2:4]

            return np.concatenate([rd, -r/np.linalg.norm(r)**3])

T = 100
N = 10000



U0 = np.array([1, 0, 0, 1])

vector_tiempos_euler, U_solucion_euler = cauchy_integration(F, U0, T, N, "euler")
vector_tiempos_rk4, U_solucion_rk4 = cauchy_integration(F, U0, T, N, "rk4")
vector_tiempos_cn, U_solucion_cn = cauchy_integration(F, U0, T, N, "crank_nicholson")

x_eu, y_eu = U_solucion_euler[:,0], U_solucion_euler[:,1]
x_rk, y_rk = U_solucion_rk4[:,0], U_solucion_rk4[:,1]
x_cn, y_cn = U_solucion_cn[:,0], U_solucion_cn[:,1]


# Estimación de error mediante Richardson


def richardson_error(F, U0,T, method, h, integrator):
      
      N1 = int(T/h)
      N2 = int(2*N1)

      vector_tiempos_1, U_solucion_1 = integrator(F, U0, T, N1, method)
      vector_tiempos_1, U_solucion_2 = integrator(F, U0, T, N2, method)

      U2_mitad = U_solucion_2[::2, :]

      error = norm(U_solucion_1[-1] - U_solucion_2[-1])

      return error 


h = T/N

errors = {}
for method in ["euler", "rk4", "crank_nicholson"]:
    e = richardson_error(F, U0, T, method, h, cauchy_integration)
    errors[method] = e

print("\nErrores estimados mediante Richardson:")
for m, e in errors.items():
    print(f"{m:18s}  Error = {e:.3e}")



# Hacemos un bucleo para ver como cambia el error en funcion de los valores de T y N


Errores estimados mediante Richardson:
euler               Error = 1.408e+00
rk4                 Error = 5.908e-08
crank_nicholson     Error = 8.842e-04
